In [1263]:
import pandas as pd
import os, re
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sqlalchemy import create_engine, types
from sqlalchemy import text 
from openpyxl import load_workbook

In [1264]:
from dotenv import dotenv_values

config = dotenv_values()

# define variables for the login
pg_user = config['POSTGRES_USER']  # align the key label with your .env file !
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']
pg_pass = config['POSTGRES_PASS']

In [1265]:
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [1266]:
my_schema = 'team_5' # update it to your schema

with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

In [1267]:
df_1= pd.read_csv('../data/Raw/World_Marriage_Dataset.csv')

In [1268]:
df_1.drop(columns=["Sr.No."], inplace=True)

In [1269]:
df_1.columns = df_1.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1270]:
df_1.rename(columns={
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "dataprocess": "data_process",
    "Data Collection (Start Year)": "data_collection_start_year",
    "Data Collection (End Year)": "data_collection_end_year",
    "Data Source": "data_source",
    "Country": "country",
    "Sex": "sex"
}, inplace=True)

In [1271]:
df_1.drop_duplicates(inplace=True)

df_1['data_collection_start_year'] = df_1['data_collection_start_year'].astype(str).str.replace(',', '').astype(int)
df_1['data_collection_end_year'] = df_1['data_collection_end_year'].astype(str).str.replace(',', '').astype(int)

In [1272]:
df_1.isnull().sum()

country                       0
age_group                     0
sex                           0
marital_status                0
data_process                  0
data_collection_start_year    0
data_collection_end_year      0
data_source                   0
dtype: int64

In [1273]:
#df_1.to_csv("cleaned_world_marriage.csv", index=False)

In [1274]:
#df_1.to_sql('world_marriage', engine, if_exists='replace', index=False)

In [1275]:
df_2 = pd.read_csv('../data/Raw/age-at-marriage-women.csv')

In [1276]:
df_2.columns = df_2.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1277]:
df_2 = df_2.drop(columns=['1005564annotations'])

df_2.rename(columns={
    "entity": "country",   
}, inplace=True)

In [1278]:
df_2.drop_duplicates(inplace=True)


In [1279]:
df_2['year'] = df_2['year'].astype(str).str.replace(',', '').astype(int)

In [1280]:
df_2.isnull().sum()

country                                0
code                                   0
year                                   0
mean_age_of_women_at_first_marriage    0
dtype: int64

In [1281]:
#df_2.to_csv("cleaned_age_at_marriage_women.csv", index=False)

In [1282]:
#df_2.to_sql('age_at_marriage_women', engine, if_exists='replace', index=False)

In [1283]:
df_3= pd.read_csv('../data/Raw/marriage-rate-per-1000-inhabitants.csv')

In [1284]:
df_3.columns = df_3.columns.str.lower() \
    .str.replace(' ', '_') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1285]:
df_3.rename(columns={
    "entity": "country",   
}, inplace=True)

In [1286]:
df_3['year'] = df_3['year'].astype(str).str.replace(',', '').astype(int)

In [1287]:
df_3.drop_duplicates(inplace=True)


In [1288]:
df_3.isnull().sum()

country                                          0
code                                             0
year                                             0
crude_marriage_rate_marriages_per_1000_people    0
dtype: int64

In [1289]:
#df_3.to_csv("cleaned_marriage-rate-per-1000-inhabitants.csv", index=False)

In [1290]:
#df_3.to_sql('married_rate_per_1000', engine, if_exists='replace', index=False)

In [1291]:
df_4= pd.read_csv('../data/Raw/marriage-rates-in-1990-vs-2020.csv')

In [1292]:
df_4.columns = df_4.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1293]:
df_4 = df_4.drop(columns=['worldregionsaccordingtoowid'])

df_4.rename(columns={
    "crudemarriageratemarriagesper1000people": "crude_marriage_rate",
    "crudemarriageratemarriagesper1000people1": "crude_marriage_rate_people1",
    "year1": "year_1",
    "entity": "country"
}, inplace=True)

In [1294]:
df_4.drop_duplicates(inplace=True)
df_4.dropna(inplace=True)

In [1295]:
df_4['year_1'] = pd.to_numeric(df_4['year_1'], errors='coerce').astype('Int64')

In [1296]:
df_4.isnull().sum()

country                        0
code                           0
year                           0
crude_marriage_rate            0
crude_marriage_rate_people1    0
year_1                         0
dtype: int64

In [1297]:
#df_4.to_csv("cleaned_marriage-rates-in-1990-vs-2020.csv", index=False)

In [1298]:
#df_4.to_sql('marriage_rates_in_1990_vs_2020', engine, if_exists='replace', index=False)

In [1299]:
df_5 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [1300]:
df_5.columns = df_5.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1301]:

df_5.rename(columns={
    "shareofbirthsoutsideofmarriageofallbirths": "share_of_births_outside_of_marriage",
    "entity": "country"
}, inplace=True)

df_5.drop_duplicates(inplace=True)

In [1302]:
df_5.isnull().sum()

country                                0
code                                   0
year                                   0
share_of_births_outside_of_marriage    0
dtype: int64

In [1303]:
#df_5.to_csv("cleaned_share-of-births-outside-marriage.csv", index=False)

In [1304]:
#df_5.to_sql('share_of_births_outside_marriage', engine, if_exists='replace', index=False)

In [1305]:
df_6 = pd.read_csv('../data/Raw/share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv')

In [1306]:
df_6.columns = df_6.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

df_6.drop_duplicates(inplace=True)
df_6.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
2,Men,NaN,19,0.8,0.6,2.0,2.5,0.7,0.3,0.1,0.0
50,Women,NaN,33,76.8,86.8,93.0,81.1,63.1,49.0,38.7,NaN
31,Men,NaN,48,92.5,91.7,92.3,81.3,69.9,NaN,NaN,NaN
34,Women,NaN,17,0.1,0.3,1.0,1.3,0.4,0.1,0.0,0.0
17,Men,NaN,34,83.2,85.5,88.5,72.4,53.7,40.9,NaN,NaN


In [1307]:
df_6 = df_6.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_6.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

In [1308]:
df_6.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [1309]:
#df_6.to_csv("cleaned_share-of-men-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [1310]:
#df_6.to_sql('men_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [1311]:
df_7 = pd.read_csv('../data/Raw/share-of-births-outside-marriage.csv')

In [1312]:
df_7.columns = df_7.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1313]:
df_7.rename(columns={
    "shareofsingleparenthouseholds": "share_of_single_parent_households",
    "entity": "country"
}, inplace=True)

df_7.drop_duplicates(inplace=True)
df_7.sample(5)

,country,code,year,shareofbirthsoutsideofmarriageofallbirths
94,Austria,AUT,2008,38.8
1849,Spain,ESP,1986,8.0
1981,Switzerland,CHE,1996,7.3
290,Costa Rica,CRI,1987,38.8
367,Croatia,HRV,2003,10.1


In [1314]:
df_7.isnull().sum()

country                                      0
code                                         0
year                                         0
shareofbirthsoutsideofmarriageofallbirths    0
dtype: int64

In [1315]:
#df_7.to_csv("cleaned_share-of-single-parent-households.csv", index=False)

In [1316]:
#df_7.to_sql('single_parent_households', engine, if_exists='replace', index=False)

In [1317]:
df_8 = pd.read_csv('../data/Raw/share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv')

In [1318]:
df_8.columns = df_8.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)

In [1319]:
df_8['code'] = df_8['code'].fillna('GBR')
df_8.sample(5)

,entity,code,year,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort,proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort
64,Women,GBR,47,84.8,91.7,95.6,87.0,75.4,NaN,NaN,NaN
9,Men,GBR,26,48.3,51.0,67.7,46.8,24.1,11.2,7.1,NaN
37,Women,GBR,20,6.8,13.3,27.0,21.5,7.6,2.8,0.9,0.4
45,Women,GBR,28,65.0,77.4,88.4,72.0,48.8,29.4,20.7,NaN
61,Women,GBR,44,84.0,91.3,95.4,86.5,74.1,NaN,NaN,NaN


In [1320]:
df_8 = df_8.drop(columns=['code','proportionsofmenorwomenwhohadevermarriedbyacertainagefor1980birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor1990birthcohort',
    'proportionsofmenorwomenwhohadevermarriedbyacertainagefor2000birthcohort'])

df_8.rename(columns={
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1900birthcohort": "1900_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1920birthcohort": "1920_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1940birthcohort": "1940_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1960birthcohort": "1960_birthcohort",
    "proportionsofmenorwomenwhohadevermarriedbyacertainagefor1970birthcohort": "1970_birthcohort",
    "entity": "sex"
}, inplace=True)

df_8.drop_duplicates(inplace=True)
df_8.sample(5)

,sex,year,1900_birthcohort,1920_birthcohort,1940_birthcohort,1960_birthcohort,1970_birthcohort
22,Men,39,88.8,89.5,90.8,77.4,62.6
57,Women,40,82.4,90.5,95.0,85.3,72.0
12,Men,29,68.3,72.0,81.0,60.6,37.4
30,Men,47,92.3,91.6,92.2,81.0,69.4
47,Women,30,71.6,82.7,90.9,76.7,55.8


In [1321]:
df_8.isnull().sum()

sex                 0
year                0
1900_birthcohort    0
1920_birthcohort    0
1940_birthcohort    0
1960_birthcohort    0
1970_birthcohort    0
dtype: int64

In [1322]:
#df_8.to_csv("cleaned_share-of-women-in-england-and-wales-who-have-ever-married-by-age.csv", index=False)

In [1323]:
#df_8.to_sql('women_in_england_and_wales_married_by_age', engine, if_exists='replace', index=False)

In [1324]:
#pip install openpyxl pywin32

In [1325]:
df_excel_1 = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')

In [1326]:
#all_sheets = pd.read_excel('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx', sheet_name=None)

In [1327]:
xls_1 = pd.ExcelFile('../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx')
print(xls_1.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']


In [1328]:
excel_1 = '../data/Raw/undesa_pd_2019_wmd_marital_status.xlsx'

# Output directory (make sure it exists)
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

# List of sheets you want to extract
sheets_to_extract = ['MARITAL_STATUS_BY_AGE', 'CURRENTLY MARRIED', 'EVER_MARRIED', 'SMAM']

In [1329]:
"""for sheet in sheets_to_extract:
    # Read just this sheet into a DataFrame
    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)
    
    # Optional: Clean the filename (replace spaces with underscores, etc.)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    
    # Save the DataFrame as CSV
    df_excel_1.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")
"""

'for sheet in sheets_to_extract:\n    # Read just this sheet into a DataFrame\n    df_excel_1 = pd.read_excel(excel_1, sheet_name=sheet)\n    \n    # Optional: Clean the filename (replace spaces with underscores, etc.)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    \n    # Save the DataFrame as CSV\n    df_excel_1.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n'

In [1330]:
xls_2 = pd.ExcelFile('../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx')
print(xls_2.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'FERTILITY INDICATORS']


In [1331]:
excel_2 = '../data/Raw/undesa_pd_2019_world_fertility_dataset.xlsx'
sheet_name = 'FERTILITY INDICATORS'
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)

df_excel_2 = pd.read_excel(excel_2, sheet_name=sheet_name)


In [1332]:
"""csv_name = sheet_name.replace(' ', '_').lower() + '.csv'
csv_path = os.path.join(output_dir, csv_name)
df_excel_2.to_csv(csv_path, index=False)
print(f"Saved: {csv_path}")
"""

'csv_name = sheet_name.replace(\' \', \'_\').lower() + \'.csv\'\ncsv_path = os.path.join(output_dir, csv_name)\ndf_excel_2.to_csv(csv_path, index=False)\nprint(f"Saved: {csv_path}")\n'

In [1333]:
xls_3 = pd.ExcelFile('../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx')
print(xls_3.sheet_names)

['INFORMATION NOTE', 'Database Field Descriptions', 'Countries', 'Regions']


In [1334]:
excel_3 = '../data/Raw/undesa_pd_ds_1970-2030_fp_rev-2024_rev.xlsx'
sheets_to_extract = ['Countries', 'Regions']
output_dir = '../data/processed/'
os.makedirs(output_dir, exist_ok=True)


In [1335]:
"""
for sheet in sheets_to_extract:
    df = pd.read_excel(excel_3, sheet_name=sheet)
    csv_name = sheet.replace(' ', '_').lower() + '.csv'
    csv_path = os.path.join(output_dir, csv_name)
    df.to_csv(csv_path, index=False)
    print(f"Saved: {csv_path}")

"""

'\nfor sheet in sheets_to_extract:\n    df = pd.read_excel(excel_3, sheet_name=sheet)\n    csv_name = sheet.replace(\' \', \'_\').lower() + \'.csv\'\n    csv_path = os.path.join(output_dir, csv_name)\n    df.to_csv(csv_path, index=False)\n    print(f"Saved: {csv_path}")\n\n'

In [1336]:
df_9 = pd.read_csv('../data/Raw/unpopulation_dataportal_20250728095844.csv')
df_9.sample(5)

,IndicatorId,IndicatorName,IndicatorShortName,Source,SourceYear,Author,LocationId,Location,Iso2,Iso3,...,AgeStart,AgeEnd,Age,CategoryId,Category,EstimateTypeId,EstimateType,EstimateMethodId,EstimateMethod,Value
21501,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,740,Suriname,SR,SUR,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,51.91
18905,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,659,Saint Kitts and Nevis,KN,KNA,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,55.56
24746,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,876,Wallis and Futuna Islands,WF,WLF,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,3,Projection,55.76
1572,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,52,Barbados,BB,BRB,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,53.44
10434,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,368,Iraq,IQ,IRQ,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,69.10


In [1337]:
df_9.columns = df_9.columns.str.lower() \
    .str.replace(' ', '') \
    .str.replace('(', '') \
    .str.replace(')', '') \
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
df_9.sample(5)

,indicatorid,indicatorname,indicatorshortname,source,sourceyear,author,locationid,location,iso2,iso3,...,agestart,ageend,age,categoryid,category,estimatetypeid,estimatetype,estimatemethodid,estimatemethod,value
19867,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,688,Serbia,RS,SRB,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,68.13
24561,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,862,Venezuela (Bolivarian Republic of),VE,VEN,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,54.14
24910,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,887,Yemen,YE,YEM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,67.58
15140,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,516,Namibia,NaN,NAM,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,50.53
24090,42,Currently married (Percent),Currently married (Percent),Estimates and Projections of Women of Reproduc...,2024,United Nations Population Division,850,United States Virgin Islands,VI,VIR,...,15,49,15-49,100,Married or in a union women,1,Model-based Estimates,2,Interpolation,37.42


In [1338]:
df_9 = df_9.drop(columns=['indicatorid','indicatorshortname',
    'source',
    'author', 'locationid', 'iso2','estimatetypeid','category','categoryid','agestart','ageend','author','ageid', 'estimatetype','variantid','sexid','timeid'])

df_9.rename(columns={
    "sourceyear": "year",
    "location": "country",
    "estimatemethodid": "estimate_method",
    "iso3": "code",
}, inplace=True)



In [1339]:
df_9.drop_duplicates(inplace=True)

In [1340]:
df_9

,indicatorname,year,country,code,time,variant,sex,age,estimate_method,estimatemethod,value
0,Currently married (Percent),2024,Afghanistan,AFG,1970,Median,Female,15-49,2,Interpolation,80.94
2,Currently married (Percent),2024,Afghanistan,AFG,1971,Median,Female,15-49,2,Interpolation,80.90
4,Currently married (Percent),2024,Afghanistan,AFG,1972,Median,Female,15-49,2,Interpolation,80.87
6,Currently married (Percent),2024,Afghanistan,AFG,1973,Median,Female,15-49,2,Interpolation,80.84
8,Currently married (Percent),2024,Afghanistan,AFG,1974,Median,Female,15-49,2,Interpolation,80.53
...,...,...,...,...,...,...,...,...,...,...,...
25078,Currently married (Percent),2024,Zambia,ZMB,2021,Median,Female,15-49,3,Projection,54.31
25080,Currently married (Percent),2024,Zambia,ZMB,2022,Median,Female,15-49,3,Projection,53.82
25082,Currently married (Percent),2024,Zambia,ZMB,2023,Median,Female,15-49,3,Projection,53.35
25084,Currently married (Percent),2024,Zambia,ZMB,2024,Median,Female,15-49,3,Projection,52.91


In [1341]:
df_9.isnull().sum()

indicatorname      0
year               0
country            0
code               0
time               0
variant            0
sex                0
age                0
estimate_method    0
estimatemethod     0
value              0
dtype: int64

In [1342]:
#df_9.to_csv("cleaned_unpopulation_dataportal.csv", index=False)

In [1343]:
#df_9.to_sql('unpopulation_dataportal', engine, if_exists='replace', index=False)

In [1344]:
df_10 = pd.read_csv('../data/processed/countries_un.csv',  header=5, low_memory=False)

In [1345]:
df_10.columns = (
    df_10.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
)
df_10.sample(10)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,dataprocess
56565,Honduras,340,Married or in-union women,1993,40-44,74.090968,82.179479,Estimate
140755,Burkina Faso,854,Married or in-union women,1987,30-34,95.003714,230.599666,Estimate
110698,Anguilla,660,Married or in-union women,2037,25-29,42.582821,0.235483,Projection
131343,Trinidad and Tobago,780,Married or in-union women,2025,15-49,41.576705,158.763767,Projection
96450,Norway,578,Married or in-union women,2038,25-29,59.317004,111.325264,Projection
85812,Montserrat,500,Married or in-union women,2004,35-39,52.306447,0.103044,Estimate
120238,Somalia,706,Married or in-union women,2014,45-49,70.429615,149.377693,Estimate
136798,The former Yugoslav Republic of Macedonia,807,Married or in-union women,1978,45-49,89.280000,49.595486,Estimate
144975,Yemen,887,Married or in-union women,2028,15-49,63.445420,7046.090366,Projection
72301,Lesotho,426,Married or in-union women,2016,40-44,62.966667,26.226561,Estimate


In [1346]:
df_10.rename(columns={
    "dataprocess": "data_process",
}, inplace=True)

df_10.drop_duplicates(inplace=True)
df_10.sample(5)

,countryorarea,isocode,indicator,year,agegroup,percentage,number,data_process
76369,Luxembourg,442,Married or in-union women,2039,20-24,12.975213,2.780329,Projection
115691,Serbia,688,Married or in-union women,2013,30-34,74.318642,188.427486,Estimate
142965,Venezuela (Bolivarian Republic of),862,Married or in-union women,2020,40-44,64.339311,607.328030,Projection
134315,Turks and Caicos Islands,796,Married or in-union women,1992,30-34,63.272858,0.346419,Estimate
143098,Venezuela (Bolivarian Republic of),862,Married or in-union women,2037,25-29,52.442962,669.655200,Projection


In [1347]:
for col in ['percentage', 'number']:
    if col in df_10.columns:
        df_10[col] = (
            df_10[col]
            .astype(str)
            .str.replace(',', '.', regex=False)
            .str.extract(r'([-+]?[0-9]*\.?[0-9]+)', expand=False)
            .astype(float)
            .round(2)
        )

In [1348]:
unnamed_cols = [col for col in df_10.columns if 'unnamed' in col.lower()]
df_10.drop(columns=unnamed_cols, inplace=True)

In [1349]:
df_10.dropna(inplace=True)

In [1350]:
df_10.isnull().sum()

countryorarea    0
isocode          0
indicator        0
year             0
agegroup         0
percentage       0
number           0
data_process     0
dtype: int64

In [1351]:
df_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145800 entries, 0 to 145799
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   countryorarea  145800 non-null  object 
 1   isocode        145800 non-null  int64  
 2   indicator      145800 non-null  object 
 3   year           145800 non-null  int64  
 4   agegroup       145800 non-null  object 
 5   percentage     145800 non-null  float64
 6   number         145800 non-null  float64
 7   data_process   145800 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 8.9+ MB


In [1352]:
#df_10.to_csv("cleaned_countries_1970_2025_un.csv", index=False)

In [1353]:
#df_10.to_sql('countries_1970_2025_un', engine, if_exists='replace', index=False)

In [1354]:
df_11 = pd.read_csv('../data/processed/currently_married_un.csv',  header=2, low_memory=False)

In [1355]:
df_11.sample(8)

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
13976,Faeroe Islands,234,2014,2014,Men,[45-49],45,49,61.17,Estimate,2014 Estimate,2090,Faeroe Islands 2014 Estimate,UNSD,NaN,NaN,NaN
35188,Nigeria,566,2003,2003,Women,[30-34],30,34,90.10,Survey,2003 DHS,1884,Nigeria 2003 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
19714,Honduras,340,2001,2001,Men,[20-24],20,24,33.50,Survey,2001 ENESF,640,Honduras 2001 Encuesta Nacional de Epidemiolog...,National statistics,1.0,NaN,NaN
8887,Comoros,174,1996,1996,Men,[50-54],50,54,96.40,Survey,1996 DHS,1707,Comoros 1996 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
8,Afghanistan,4,1972,1974,Men,[55-59],55,59,91.50,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
28534,Liechtenstein,438,2000,2000,Men,[50-54],50,54,81.92,Estimate,2000 Estimate,2145,Liechtenstein 2000 Estimate,UNSD,NaN,NaN,NaN
34664,New Zealand,554,2006,2006,Men,[20-24],20,24,4.44,Census,2006 Census,1306,New Zealand 2006 Census,UNSD,NaN,Data randomly rounded to protect confidentiali...,NaN
24924,Italy,380,2007,2007,Women,[45-49],45,49,80.90,Estimate,2007 Estimate,2128,Italy 2007 Estimate,UNSD,NaN,NaN,NaN


In [1356]:
df_11.columns = (
    df_11.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_11.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
28757,Liechtenstein,438,2009,2009,Women,[75+],75,999,26.52,Estimate,2009 Estimate,2145,Liechtenstein 2009 Estimate,UNSD,NaN,NaN,NaN
36757,Norway,578,2018,2018,Women,[70-74],70,74,57.83,Estimate,2018 Estimate,2180,Norway 2018 Estimate,UNSD,1.0,NaN,NaN
14201,Fiji,242,1986,1986,Women,[60-64],60,64,58.19,Census,1986 Census,1361,Fiji 1986 Census,UNSD,NaN,NaN,NaN
30857,Marshall Islands,584,1988,1988,Women,[60-64],60,64,65.84,Census,1988 Census,2499,Marshall Islands 1988 Census,UNSD,NaN,NaN,NaN
21810,Iceland,352,1997,1997,Women,[20-24],20,24,29.89,Estimate,1997 Estimate,2121,Iceland 1997 Estimate,UNSD,1.0,NaN,NaN
47229,Sweden,752,2007,2007,Men,[15-19],15,19,0.08,Estimate,2007 Estimate,2227,Sweden 2007 Estimate,UNSD,NaN,NaN,NaN
6686,Canada,124,2001,2001,Women,[60-64],60,64,70.06,Estimate,2001 Estimate,2061,Canada 2001 Estimate,UNSD,1.0,NaN,NaN
21148,Hungary,348,2017,2017,Women,[70-74],70,74,39.55,Estimate,2017 Estimate,2120,Hungary 2017 Estimate,UNSD,1.0,NaN,NaN


In [1357]:
df_11 = df_11.drop(columns = ['datacataloglongname', 'datacatalogid', 'yearstart' , 'yearend', 'noteondata', 'noteoncountryandpopulation', 'including_consensual_unions'])

df_11.rename(columns={
    "agestart": "age_start",
    "countryorarea": "country",
    "datasource": "data_source",
    "datavalue" : "data_value"
}, inplace=True)

df_11.sample(10)

,country,isocode,sex,agegroup,age_start,ageend,data_value,dataprocess,datacatalogshortname,data_source
18267,Greenland,304,Women,[75+],75,999,18.66,Census,2000 Census,UNSD
24062,Isle of Man,833,Men,[45-49],45,49,61.72,Census,2011 Census,UNSD
10919,Czechia,203,Men,[70-74],70,74,77.80,Estimate,2013 Estimate,UNSD
34358,Netherlands,528,Men,[75+],75,999,69.10,Estimate,2018 Estimate,UNSD
19687,Honduras,340,Men,[20-24],20,24,38.52,Census,2001 Census,UNSD
11817,Denmark,208,Women,[50-54],50,54,66.33,Estimate,2009 Estimate,UNSD
10917,Czechia,203,Men,[60-64],60,64,73.74,Estimate,2013 Estimate,UNSD
47203,Sweden,752,Men,[25-29],25,29,11.48,Estimate,2006 Estimate,UNSD
8788,Colombia,170,Men,[70-74],70,74,74.77,Survey,2015 DHS,DHS_HH
49773,Tunisia,788,Men,[55-59],55,59,93.41,Census,1975 Census,UNSD


In [1358]:
df_11.drop_duplicates(inplace=True)

In [1359]:
df_11.isnull().sum()

country                 0
isocode                 0
sex                     0
agegroup                0
age_start               0
ageend                  0
data_value              0
dataprocess             0
datacatalogshortname    0
data_source             0
dtype: int64

In [1360]:
#df_11.to_csv("cleaned_currently_married_un.csv", index=False)

In [1361]:
#df_11.to_sql('currently_married_un', engine, if_exists='replace', index=False)

In [1362]:
df_12 = pd.read_csv('../data/processed/ever_married_un.csv', header= 2, low_memory = False)
df_12.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,AgeGroup,AgeStart,AgeEnd,DataValue,DataProcess,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Data,Note on Country and Population
0,Afghanistan,4,1972,1974,Men,[15-19],15,19,7.7,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,[20-24],20,24,32.6,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,[25-29],25,29,61.4,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,[30-34],30,34,83.0,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,[35-39],35,39,91.2,Survey,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN


In [1363]:
df_12.columns = (
    df_12.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
    )
df_12.sample(8)

,countryorarea,isocode,yearstart,yearend,sex,agegroup,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteondata,noteoncountryandpopulation
42954,Saint Vincent and the Grenadines,670,2012,2012,Men,[70-74],70,74,81.38,Census,2012 Census,5277,Saint Vincent and the Grenadines 2012 Census,National statistics,1.0,NaN,NaN
21654,Hungary,348,1990,1990,Men,[55-59],55,59,96.22,Census,1990 Census,827,Hungary 1990 Census,UNSD,1.0,NaN,NaN
3720,Benin,204,2011,2012,Women,[45-49],45,49,97.90,Survey,2011-2012 DHS,4493,Benin 2011-2012 Demographic and Health Survey,DHS_HH,NaN,NaN,NaN
26291,Italy,380,1991,1991,Men,[40-44],40,44,88.24,Census,1991 Census,359,Italy 1991 Census,US Census Bureau,NaN,NaN,NaN
29337,Lesotho,426,1996,1996,Women,[75+],75,999,97.50,Census,1996 Census,1066,Lesotho 1996 Census,INED,NaN,NaN,NaN
37004,Niger,562,1998,1998,Women,[20-24],20,24,88.90,Survey,1998 DHS,1680,Niger 1998 Demographic and Health Survey,DHS_STATcompiler,1.0,NaN,NaN
699,Antigua and Barbuda,28,2001,2001,Men,[20-24],20,24,10.00,Census,2001 Census,2277,Antigua and Barbuda 2001 Census,UNSD,1.0,NaN,NaN
52431,Turkmenistan,795,2015,2015,Women,[40-44],40,44,97.54,Survey,2015 MICS,5731,Turkmenistan 2015 Multiple Indicator Cluster S...,MICS,1.0,NaN,NaN


In [1364]:
df_12 = df_12.drop(columns = ['yearstart', 'yearend', 'datacatalogshortname', 'datacatalogid', 'datacataloglongname', 'including_consensual_unions', 'noteondata', 'noteoncountryandpopulation'])

df_12.rename(columns={
    "agestart": "age_start",
    "ageend": "age_end",
    "countryorarea": "country"
}, inplace=True)
df_12.sample(8)

,country,isocode,sex,agegroup,age_start,age_end,datavalue,dataprocess,datasource
38097,Norway,578,Men,[70-74],70,74,90.03,Estimate,UNSD
18156,Germany,276,Women,[25-29],25,29,54.37,Census,Eurostat
3845,Bermuda,60,Men,[60-64],60,64,91.86,Census,UNSD
42498,Rwanda,646,Men,[25-29],25,29,63.40,Survey,DHS_HH
6601,Canada,124,Women,[15-19],15,19,5.20,Estimate,UNSD
42255,Rwanda,646,Women,[30-34],30,34,98.26,Census,UNSD
3110,Belgium,56,Men,[30-34],30,34,87.48,Census,UNSD
55562,Zimbabwe,716,Men,[65-69],65,69,99.70,Survey,UNSD


In [1365]:
df_12.dropna(inplace=True)

In [1366]:
df_12.isnull().sum()

country        0
isocode        0
sex            0
agegroup       0
age_start      0
age_end        0
datavalue      0
dataprocess    0
datasource     0
dtype: int64

In [1367]:
#df_12.to_csv("cleaned_ever_married_un.csv", index=False)

In [1368]:
#df_12.to_sql('ever_married_un', engine, if_exists= 'replace', index= False)

In [1369]:
df_13 = pd.read_csv('../data/processed/fertility_indicators_un.csv', header=6, low_memory=False)
df_13.head()

,Country or Area,Country or Area Code,Age Group,Indicator,Date,Value,Series,DataType,Data Source Type,Survey Programme,Data Source Inventory ID,Data Source Name,Data Source Name (short),Data Source Start Year,Data Source End Year,Reference,Reference Year
0,Afghanistan,4,[Total],TFR,1964.977051,7.966653,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
1,Afghanistan,4,[Total],TFR,1965.977051,8.212275,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
2,Afghanistan,4,[Total],TFR,1966.977051,8.317603,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
3,Afghanistan,4,[Total],TFR,1967.977051,8.225812,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012
4,Afghanistan,4,[Total],TFR,1968.977051,8.068459,"1979 Census,Reverse survival methods,Computed",Reverse survival method,Census,Census,280,Afghanistan 1979 Census,1979 Census,1979,1979,United Nations Population Division,2012


In [1370]:
df_13.columns = (df_13.columns
        .str.lower()
        .str.strip()
        .str.replace(' ', '')
        .str.replace('(', '')
        .str.replace(')', '')
        .str.replace('[^0-9a-zA-Z_]','' ,regex= True)
        )

df_13.sample(6)

,countryorarea,countryorareacode,agegroup,indicator,date,value,series,datatype,datasourcetype,surveyprogramme,datasourceinventoryid,datasourcename,datasourcenameshort,datasourcestartyear,datasourceendyear,reference,referenceyear
9791,Brazil,76,[15-19],ASFR1519,1994.500000,64.98000,"Estimates,Direct,SINASC & IBGE,2050-16-41293",Direct,Estimate,Estimate,2050,All sources of estimates,Estimates,1994,1994,SINASC births and IBGE population,2017
40022,Kuwait,414,[Total],TFR,1992.500000,5.32000,"Estimates,Direct,DYB,2137-16-56",Direct,Estimate,Estimate,2137,All sources of estimates,Estimates,1992,1992,Demographic Yearbook,2003
17119,Costa Rica,188,[30-34],ASFR3034,1965.501370,256.00000,"Estimates, Fertility data (adjusted), Rosero-B...",Fertility data (adjusted),Estimate,Estimate,2075,All sources of estimates,Estimates,1965,1965,"La situación demográfica en Costa Rica, Poblac...",2023
77579,Viet Nam,704,[Total],MAC,1995.747192,27.64388,"1999 Census,Own-children estimate,Computed",Own-children method,Census,Census,951,Viet Nam 1999 Census,1999 Census,1999,1999,United Nations Population Division,2013
8906,Bosnia and Herzegovina,70,[20-24],ASFR2024,1955.500000,217.75400,"Estimates,Fertility data (Adjusted),HFC-ODE,20...",Fertility data (adjusted),Estimate,Estimate,2048,All sources of estimates,Estimates,1955,1955,European Demographic Observatory (ODE). Data c...,2011
30881,Guinea,324,[30-34],ASFR3034,1997.954956,214.00000,"1999 DHS,Direct,DHS,1739-16-39167",Direct,Survey,DHS,1739,Guinea 1999 Demographic and Health Survey,1999 DHS,1999,1999,DHS Statcompiler,2012


In [1371]:
df_13 = df_13.drop(columns=['countryorareacode','indicator','datasourceinventoryid','surveyprogramme','series','datasourcename','reference','referenceyear'])

df_13.replace({
    "agegroup": "age_group",
    "countryorarea": "country",
    "datatype": "data_type",
},inplace=True)

In [1372]:
df_13['date'] = df_13['date'].astype(int)
df_13['value'] = df_13['value'].round(2)
df_13.sample(12)

,countryorarea,agegroup,date,value,datatype,datasourcetype,datasourcenameshort,datasourcestartyear,datasourceendyear
21913,Ecuador,[45-49],1969,33.96,Extrapolated from Truncated Birth Histories,Survey,1987 DHS,1987,1987
14253,China,[25-29],1987,176.00,Birth histories,Survey,1988 Two-per-Thousand FS,1988,1988
15174,"China, Hong Kong SAR",[Total],1968,29.41,Fertility data (adjusted),Estimate,Estimates,1968,1968
8982,Bosnia and Herzegovina,[40-44],1963,42.19,Fertility data (adjusted),Estimate,Estimates,1963,1963
52449,Nicaragua,[35-39],1989,128.00,Direct,Survey,2001 DHS,2001,2001
26015,France,[15-19],1995,9.68,Fertility data (adjusted),Estimate,Estimates,1995,1995
79340,Zimbabwe,[40-44],1982,151.32,Extrapolated from Truncated Birth Histories,Survey,1994 DHS,1994,1994
73166,United Arab Emirates,[Total],1986,5.91,Recent births,Survey,1987-1988 CHS,1987,1988
42593,Liberia,[Total],1971,5.59,P/F Ratio method (Feeney),Panel,1971 PGS,1970,1971
19811,Czechia,[35-39],1958,27.68,Direct,Register,Register,1958,1958


In [1373]:
#df_13.to_csv("cleaned_fertility_indicators.csv", index=False)

In [1374]:
#df_13.to_sql('fertility_indicators_un',engine, if_exists='replace', index=False)

In [1375]:
df_14 = pd.read_csv('../data/processed/marital_status_by_age_un.csv', header= 2, low_memory=False)
df_14.head()

,Country or area,ISO code,YearStart,YearEnd,Sex,MaritalStatus,Non-standard_AgeGroups,Series_contains_Non-standard_AgeGroups,AgeGroup,AgeStart,...,DataCatalog ShortName,DataCatalog ID,DataCatalog LongName,Data Source,Including_consensual_unions,Note on Age groups,Note on Marital Status,Note on Data,Note on Country and Population,Note Other
0,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[15-19],15,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[20-24],20,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[25-29],25,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[30-34],30,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,4,1972,1974,Men,Divorced,NaN,NaN,[35-39],35,...,1972-1974 NDFGS,160,Afghanistan 1972-1974 National Demographic and...,National statistics,NaN,NaN,NaN,NaN,NaN,NaN


In [1376]:
df_14.columns= (df_14.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '' , regex=True)  
    )
df_14.sample(5)

,countryorarea,isocode,yearstart,yearend,sex,maritalstatus,nonstandard_agegroups,series_contains_nonstandard_agegroups,agegroup,agestart,...,datacatalogshortname,datacatalogid,datacataloglongname,datasource,including_consensual_unions,noteonagegroups,noteonmaritalstatus,noteondata,noteoncountryandpopulation,noteother
211228,San Marino,674,1984,1984,Women,Married,NaN,NaN,[65-69],65,...,1984 Estimate,2208,San Marino 1984 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
7740,Australia,36,2006,2006,Men,Widowed,NaN,NaN,[30-34],30,...,2006 Estimate,2037,Australia 2006 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
126190,Israel,376,2015,2015,Women,Divorced,NaN,NaN,[65-69],65,...,2015 Estimate,2127,Israel 2015 Estimate,UNSD,NaN,NaN,NaN,NaN,NaN,NaN
245238,Togo,768,1998,1998,Men,Married,NaN,NaN,[45-49],45,...,1998 DHS,1735,Togo 1998 Demographic and Health Survey,DHS_STATcompiler,NaN,NaN,NaN,NaN,NaN,NaN
167082,Namibia,516,2011,2011,Women,Widowed,NaN,NaN,[65-69],65,...,2011 Census,4719,Namibia 2011 Census,UNSD,NaN,NaN,NaN,NaN,NaN,NaN


In [1377]:
df_14 = df_14.drop(columns=['datacataloglongname', 'noteondata', 'noteoncountryandpopulation','noteonagegroups', 'noteother',
                             'including_consensual_unions','isocode', 'datacatalogid', 'noteonmaritalstatus', 'series_contains_nonstandard_agegroups','nonstandard_agegroups'])

df_14.rename(columns={
    "countryorarea": "country",
    "agegroup": "age_group",
    "maritalstatus": "marital_status",
    "yearstart": "year_start",
    "yearend": "year_end",
    }, inplace =True
    )

df_14.sample(10)

,country,year_start,year_end,sex,marital_status,age_group,agestart,ageend,datavalue,dataprocess,datacatalogshortname,datasource
228589,Spain,2011,2011,Men,Widowed,[45-49],45,49,0.55,Census,2011 Census,Eurostat
227054,South Africa,2016,2016,Women,Widowed,[25-29],25,29,0.40,Survey,2016 DHS,DHS_STATcompiler
152461,Maldives,1977,1977,Men,Married,[75+],75,999,52.59,Census,1977 Census,UNSD
187679,Pakistan,2000,2001,Men,Single,[25-29],25,29,44.10,Survey,2000-2001 RHFPS,National statistics
149636,Luxembourg,2011,2011,Men,Single,[25-29],25,29,67.98,Census,2011 Census,Eurostat
165366,Mozambique,2017,2017,Men,Widowed,[20-24],20,24,0.16,Census,2017 Census,National statistics
269858,Zambia,2013,2014,Women,Not living together,[35-39],35,39,2.50,Survey,2013-2014 DHS,DHS_STATcompiler
233922,Sweden,1982,1982,Men,Divorced,[70-74],70,74,5.32,Estimate,1982 Estimate,UNSD
144677,Liechtenstein,2001,2001,Women,Married,[25-29],25,29,39.51,Estimate,2001 Estimate,UNSD
11298,Azerbaijan,2014,2014,Men,Single,[45-49],45,49,2.75,Estimate,2014 Estimate,UNSD


In [1378]:
df_14.drop_duplicates(inplace=True)
df_14.isnull().sum()

country                 0
year_start              0
year_end                0
sex                     0
marital_status          0
age_group               0
agestart                0
ageend                  0
datavalue               0
dataprocess             0
datacatalogshortname    0
datasource              0
dtype: int64

In [1379]:
#df_14.to_csv("cleaned_marital_status_by_age_un.csv", index=False)

In [1380]:
#df_14.to_sql('marital_status_by_age_un', engine, if_exists='replace', index=False)

In [1381]:
df_15 = pd.read_csv('../data/processed/regions_un.csv', header=5, low_memory= False)
df_15.head(10)

,Region and subregion,ISO code,Regional Classification,Indicator,Year,AgeGroup,Percentage,Number,DataProcess
0,World,900,M49,Married or in-union women,1970,15-19,22.576683,7.186782e+04,Estimate
1,World,900,M49,Married or in-union women,1970,20-24,63.802057,1.628604e+05,Estimate
2,World,900,M49,Married or in-union women,1970,25-29,87.174827,1.826811e+05,Estimate
3,World,900,M49,Married or in-union women,1970,30-34,90.825027,1.791214e+05,Estimate
4,World,900,M49,Married or in-union women,1970,35-39,90.284386,1.615263e+05,Estimate
5,World,900,M49,Married or in-union women,1970,40-44,86.483531,1.393344e+05,Estimate
6,World,900,M49,Married or in-union women,1970,45-49,82.680237,1.160884e+05,Estimate
7,World,900,M49,Married or in-union women,1970,15-49,69.379111,1.013480e+06,Estimate
8,World,900,M49,Married or in-union women,1971,15-19,22.630416,7.412762e+04,Estimate
9,World,900,M49,Married or in-union women,1971,20-24,63.613178,1.700873e+05,Estimate


In [1382]:
df_15.columns = (df_15.columns
    .str.lower()
    .str.strip()
    .str.replace(' ', '')
    .str.replace('(','')
    .str.replace(')', '')
    .str.replace('[^0-9a-zA-Z_]', '', regex=True)
    )
df_15.sample(6)

,regionandsubregion,isocode,regionalclassification,indicator,year,agegroup,percentage,number,dataprocess
23039,Developed countries,901,Development group,Married or in-union women,2014,15-49,55.504750,159485.939781,Estimate
7585,Southern Africa,913,M49,Married or in-union women,2027,20-24,9.734727,600.856049,Projection
27475,High-income countries,1503,Income group,Married or in-union women,2002,30-34,73.738313,34679.830812,Estimate
10057,Eastern Asia,906,SDG-M49,Married or in-union women,2012,20-24,27.681538,34300.246003,Estimate
3194,Eastern and South-Eastern Asia,753,SDG,Married or in-union women,2045,25-29,44.862935,60260.613866,Projection
3704,Oceania excluding Australia and New Zealand,543,SDG,Married or in-union women,2028,15-19,11.196718,156.817427,Projection


In [1383]:
df_15 = df_15.drop(columns=['regionalclassification'])

df_15.rename(columns={
    "regionandsubregion": "region",
    "isocode": "iso_code",
    "agegroup": "age_group",
    "dataprocess": "process"
}, inplace=True)

df_15.sample(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
20100,Australia and New Zealand,927,Married or in-union women,2011,25-29,55.484615,534.623118,Estimate
8675,Asia,935,Married or in-union women,2001,30-34,91.317897,274275.310608,Estimate
1032,Sub-Saharan Africa,202,Married or in-union women,2018,15-19,20.216002,22758.569686,Estimate
14787,Southern Europe,925,Married or in-union women,2036,30-34,50.144450,1941.893218,Projection
19065,Oceania,909,Married or in-union women,2004,20-24,43.597001,729.790690,Estimate
390,World,900,Married or in-union women,2018,45-49,84.030211,359490.586880,Estimate
4535,Europe and Northern America,513,Married or in-union women,2050,15-49,46.175003,103627.968218,Projection
18614,Northern America,905,Married or in-union women,2028,45-49,69.966613,8664.000990,Projection
10323,Eastern Asia,906,Married or in-union women,2045,30-34,70.499872,63846.436104,Projection
8754,Asia,935,Married or in-union women,2011,25-29,79.738823,272846.933926,Estimate


In [1384]:
df_15.dropna(inplace=True)
df_15.isnull().sum()

region        0
iso_code      0
indicator     0
year          0
age_group     0
percentage    0
number        0
process       0
dtype: int64

In [1385]:
print(df_15['number'] % 1 != 0)

0        True
1        True
2        True
3        True
4        True
         ... 
28507    True
28508    True
28509    True
28510    True
28511    True
Name: number, Length: 28512, dtype: bool


In [1386]:
df_15['percentage'] = df_15['percentage'].round(2)
df_15['number'] = df_15['number'].astype(int)
df_15.head(10)

,region,iso_code,indicator,year,age_group,percentage,number,process
0,World,900,Married or in-union women,1970,15-19,22.58,71867,Estimate
1,World,900,Married or in-union women,1970,20-24,63.80,162860,Estimate
2,World,900,Married or in-union women,1970,25-29,87.17,182681,Estimate
3,World,900,Married or in-union women,1970,30-34,90.83,179121,Estimate
4,World,900,Married or in-union women,1970,35-39,90.28,161526,Estimate
5,World,900,Married or in-union women,1970,40-44,86.48,139334,Estimate
6,World,900,Married or in-union women,1970,45-49,82.68,116088,Estimate
7,World,900,Married or in-union women,1970,15-49,69.38,1013479,Estimate
8,World,900,Married or in-union women,1971,15-19,22.63,74127,Estimate
9,World,900,Married or in-union women,1971,20-24,63.61,170087,Estimate


In [1387]:
#df_15.to_csv('cleaned_regions_un.csv', index=False)



In [1388]:
#df_15.to_sql('regions_un', engine, if_exists='replace',index=False)

In [1389]:
#pip install "xlrd==1.2.0"


In [1390]:
excel_1_1 = pd.read_excel('../data/Raw/OECD/SF_1_1_Family_size_and_composition.xlsx')
excel_1_1.head(10)

,Unnamed: 0,"Chart SF1.1.A. Average size of households by household type, 2024a",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,"Data for Chart SF1.1.A. Average size of households by household type, 2024a",Unnamed: 13,Unnamed: 14,Unnamed: 15
0,NaN,"Mean average number of people per household, b...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mean average number of people per household, b...",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All households,Couple households with children,Single parent households with children
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,3.56,4.08,2.76
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Costa Rica,3.462513,4.372663,3.443867
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Türkiye,3.2,4.1,2.8
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Israel,3.19,4.649476,2.863297
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbia,3.100732,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slovak Republic,3.1,3.8,2.5
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chile,2.8,NaN,NaN


In [1391]:
file_path  = '../data/Raw/OECD/SF_1_1_Family_size_and_composition.xlsx'
output_csv = '../data/processed/household_oecd_combined.csv'

xls = pd.ExcelFile(file_path)

# ----------------- yardımcılar -----------------
def find_row_with_value(df, pattern):
    """"Dataframe first row that matches the pattern."""
    pat = re.compile(pattern, re.IGNORECASE)
    for i in range(df.shape[0]):
        row = df.iloc[i].astype(str).tolist()
        if any(pat.search(str(x)) for x in row):
            return i
    return 0

def pick_country_column(df):
    """'Unnamed' kolonlar arasından ülke adlarının olduğu en dolu olanı seç."""
    candidates = [c for c in df.columns if str(c).lower().startswith("unnamed")]
    if not candidates:
        return df.columns[0]
    return max(candidates, key=lambda c: df[c].astype(str).ne("nan").sum())

# ----------------- Sheet 1: Chart SF1.1.A -----------------
# Automatical finding of header row
_chart_raw = pd.read_excel(xls, sheet_name='Chart SF1.1.A', header=None)
chart_header_row = find_row_with_value(_chart_raw, r"\bAll households\b")

chart_df = pd.read_excel(xls, sheet_name='Chart SF1.1.A', header=chart_header_row)
country_col_chart = pick_country_column(chart_df)

chart_df = chart_df.rename(columns={country_col_chart: "country"})
rename_chart = {}
for c in chart_df.columns:
    lc = str(c).lower()
    if "all households" in lc:
        rename_chart[c] = "avg_size_all"
    elif "couple households with children" in lc:
        rename_chart[c] = "avg_size_couple_with_children"
    elif "single parent households with children" in lc:
        rename_chart[c] = "avg_size_single_parent_with_children"

chart_df = chart_df[["country"] + list(rename_chart.keys())].rename(columns=rename_chart)
chart_df["country"] = chart_df["country"].astype(str).str.strip()
for col in rename_chart.values():
    chart_df[col] = pd.to_numeric(chart_df[col], errors="coerce")
chart_df = chart_df.dropna(subset=[c for c in chart_df.columns if c != "country"], how="all")

# ----------------- Sheet 2: Table SF1.1.A (çok satırlı başlık) -----------------
_table_a_raw = pd.read_excel(xls, sheet_name='Table SF1.1.A', header=None)
table_a_header_row_top = find_row_with_value(_table_a_raw, r"Couple households")
table_a = pd.read_excel(xls, sheet_name='Table SF1.1.A',
                        header=[table_a_header_row_top, table_a_header_row_top+1])

# Ülke kolonu: en dolu Unnamed çifti
mi_cols = table_a.columns
unnamed_pairs = [c for c in mi_cols
                 if str(c[0]).lower().startswith("unnamed")
                 and str(c[1]).lower().startswith("unnamed")]
country_multi_col = max(unnamed_pairs, key=lambda c: table_a[c].astype(str).ne("nan").sum())

def match_best(mi_cols, top_label, sub_keyword):
    """Find best matching column in multi-index columns."""
    for c in mi_cols:
        top = str(c[0]).strip()
        sub = str(c[1]).strip()
        if top == top_label:
            if not sub_keyword:
                return c  # Get first match (e.g., "Single person" / "Other" sub-headings)
            if sub_keyword.lower() in sub.lower():
                return c
    return None

specs = [
    ("Couple households:", "Total",             "share_couple_total"),
    ("Couple households:", "With children",     "share_couple_with_children"),
    ("Couple households:", "Without children",  "share_couple_without_children"),
    ("Single parent households:", "Total",      "share_single_parent_total"),
    ("Single parent households:", "Single mother households", "share_single_mother"),
    ("Single parent households:", "Single father households", "share_single_father"),
    ("Single person households", "",            "share_single_person"),
    ("Other household types", "",              "share_other_types"),
]

selected_cols = {country_multi_col: "country"}
for top, sub, name in specs:
    c = match_best(mi_cols, top, sub)
    if c is not None:
        selected_cols[c] = name

table_a_clean = table_a.loc[:, list(selected_cols.keys())].copy()
table_a_clean.columns = list(selected_cols.values())
table_a_clean["country"] = table_a_clean["country"].astype(str).str.strip()
for col in [c for c in table_a_clean.columns if c != "country"]:
    table_a_clean[col] = pd.to_numeric(table_a_clean[col], errors="coerce")
table_a_clean = table_a_clean.dropna(subset=[c for c in table_a_clean.columns if c != "country"], how="all")

# ----------------- Sheet 3: Table SF1.1.B -----------------
_table_b_raw = pd.read_excel(xls, sheet_name='Table SF1.1.B', header=None)
table_b_header_row = find_row_with_value(_table_b_raw, r"^0\s*children$")
table_b = pd.read_excel(xls, sheet_name='Table SF1.1.B', header=table_b_header_row)

country_col_b = pick_country_column(table_b)
table_b = table_b.rename(columns={country_col_b: "country"})

rename_b = {}
for c in table_b.columns:
    lc = str(c).lower()
    if lc == "country":
        continue
    if re.search(r"^0\s*children", lc):
        rename_b[c] = "share_hh_0_children"
    elif re.search(r"^1\s*child", lc):
        rename_b[c] = "share_hh_1_child"
    elif re.search(r"^2\s*children", lc):
        rename_b[c] = "share_hh_2_children"
    elif "3 or more children" in lc:
        rename_b[c] = "share_hh_3plus_children"
    elif ("under 6" in lc) or ("under six" in lc):
        rename_b[c] = "share_hh_with_child_under6"

table_b = table_b[["country"] + list(rename_b.keys())].rename(columns=rename_b)
table_b["country"] = table_b["country"].astype(str).str.strip()
for col in rename_b.values():
    table_b[col] = pd.to_numeric(table_b[col], errors="coerce")
table_b = table_b.dropna(subset=[c for c in table_b.columns if c != "country"], how="all")

# ----------------- Together+ Save -----------------
combined = (
    chart_df
    .merge(table_a_clean, on="country", how="outer")
    .merge(table_b,       on="country", how="outer")
    .sort_values("country")
    .reset_index(drop=True)
)

# Save the combined DataFrame to CSV
Path(output_csv).parent.mkdir(parents=True, exist_ok=True)
combined.to_csv(output_csv, index=False)
print(f"✅ Saved: {output_csv}")
display(combined.head(12))

✅ Saved: ../data/processed/household_oecd_combined.csv


,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total,share_couple_with_children,share_couple_without_children,share_single_parent_total,share_single_mother,share_single_father,share_single_person,share_other_types,share_hh_0_children,share_hh_1_child,share_hh_2_children,share_hh_3plus_children
0,Australia,2.527580,3.932863,2.775636,55.926052,29.904701,26.021351,10.373549,NaN,NaN,25.124159,8.576123,NaN,NaN,NaN,NaN
1,Austria,2.200000,3.800000,2.500000,48.927373,21.129493,27.797880,5.628256,4.779580,0.848677,38.337314,7.107057,77.781493,10.524053,8.574977,3.119476
2,Belgium,2.200000,3.900000,2.600000,52.219429,23.979842,28.239587,7.423107,6.077464,1.345643,35.502304,4.855161,73.974236,11.757354,10.153817,4.112671
3,Bulgaria,2.200000,3.500000,2.300000,40.303059,16.350410,23.952650,4.603051,3.875726,0.727325,35.809580,19.284309,78.210294,12.932273,7.480467,1.376966
4,Canada,2.425303,NaN,NaN,50.919441,25.300814,25.618627,8.715567,NaN,NaN,29.347961,11.017031,NaN,NaN,NaN,NaN
5,Chile,2.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Columbia,3.100732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Costa Rica,3.462513,4.372663,3.443867,52.441873,38.147069,14.294803,10.548101,9.489556,1.058545,11.270909,25.739118,30.290198,23.077315,24.608947,22.023540
8,Croatia,2.600000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
9,Cyprus,2.500000,3.700000,2.400000,56.920863,27.422456,29.498128,6.165260,4.936008,1.229252,24.490862,12.426368,71.356113,13.881474,11.665777,3.096636


In [1392]:
df_16 = pd.read_csv('../data/processed/household_oecd_combined.csv')
df_16

,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total,share_couple_with_children,share_couple_without_children,share_single_parent_total,share_single_mother,share_single_father,share_single_person,share_other_types,share_hh_0_children,share_hh_1_child,share_hh_2_children,share_hh_3plus_children
0,Australia,2.527580,3.932863,2.775636,55.926052,29.904701,26.021351,10.373549,NaN,NaN,25.124159,8.576123,NaN,NaN,NaN,NaN
1,Austria,2.200000,3.800000,2.500000,48.927373,21.129493,27.797880,5.628256,4.779580,0.848677,38.337314,7.107057,77.781493,10.524053,8.574977,3.119476
2,Belgium,2.200000,3.900000,2.600000,52.219429,23.979842,28.239587,7.423107,6.077464,1.345643,35.502304,4.855161,73.974236,11.757354,10.153817,4.112671
3,Bulgaria,2.200000,3.500000,2.300000,40.303059,16.350410,23.952650,4.603051,3.875726,0.727325,35.809580,19.284309,78.210294,12.932273,7.480467,1.376966
4,Canada,2.425303,NaN,NaN,50.919441,25.300814,25.618627,8.715567,NaN,NaN,29.347961,11.017031,NaN,NaN,NaN,NaN
5,Chile,2.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Columbia,3.100732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Costa Rica,3.462513,4.372663,3.443867,52.441873,38.147069,14.294803,10.548101,9.489556,1.058545,11.270909,25.739118,30.290198,23.077315,24.608947,22.023540
8,Croatia,2.600000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
9,Cyprus,2.500000,3.700000,2.400000,56.920863,27.422456,29.498128,6.165260,4.936008,1.229252,24.490862,12.426368,71.356113,13.881474,11.665777,3.096636


In [1393]:
# To numeric columns, except 'country'
metric_cols = [c for c in df_16.columns if c != 'country']
for c in metric_cols:
    df_16[c] = pd.to_numeric(df_16[c], errors='coerce')

# 3) Long/tidy verse format
tidy_16 = (df_16
           .melt(id_vars='country',
                 value_vars=metric_cols,
                 var_name='metric',
                 value_name='value')
           .dropna(subset=['value'])
           .sort_values(['country','metric'])
           .reset_index(drop=True))

# (opsiyonel) 
tidy_16['unit'] = tidy_16['metric'].apply(lambda m: 'persons' if m.startswith('avg_size') else 'percent')
def metric_group(m):
    if m.startswith('avg_size'): return 'average_size'
    if m.startswith('share_hh_'): return 'children_count_distribution'
    if m.startswith('share_couple'): return 'couple_households'
    if m.startswith('share_single_parent'): return 'single_parent_households'
    if m in ('share_single_mother','share_single_father'): return 'single_parent_gender'
    if m == 'share_single_person': return 'single_person_households'
    if m == 'share_other_types': return 'other_types'
    return 'other'
tidy_16['group'] = tidy_16['metric'].map(metric_group)

# 4) Gerekirse tidy üstünde oynadıktan sonra tekrar WIDE'a dön ve df_16'ya geri yaz
df_16 = (tidy_16.pivot_table(index='country', columns='metric', values='value', aggfunc='first')
         .reset_index()
         .reindex(columns=['country'] + metric_cols)   # orijinal kolon sırası
        )

In [1394]:
df_16

metric,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total,share_couple_with_children,share_couple_without_children,share_single_parent_total,share_single_mother,share_single_father,share_single_person,share_other_types,share_hh_0_children,share_hh_1_child,share_hh_2_children,share_hh_3plus_children
0,Australia,2.527580,3.932863,2.775636,55.926052,29.904701,26.021351,10.373549,NaN,NaN,25.124159,8.576123,NaN,NaN,NaN,NaN
1,Austria,2.200000,3.800000,2.500000,48.927373,21.129493,27.797880,5.628256,4.779580,0.848677,38.337314,7.107057,77.781493,10.524053,8.574977,3.119476
2,Belgium,2.200000,3.900000,2.600000,52.219429,23.979842,28.239587,7.423107,6.077464,1.345643,35.502304,4.855161,73.974236,11.757354,10.153817,4.112671
3,Bulgaria,2.200000,3.500000,2.300000,40.303059,16.350410,23.952650,4.603051,3.875726,0.727325,35.809580,19.284309,78.210294,12.932273,7.480467,1.376966
4,Canada,2.425303,NaN,NaN,50.919441,25.300814,25.618627,8.715567,NaN,NaN,29.347961,11.017031,NaN,NaN,NaN,NaN
5,Chile,2.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Columbia,3.100732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Costa Rica,3.462513,4.372663,3.443867,52.441873,38.147069,14.294803,10.548101,9.489556,1.058545,11.270909,25.739118,30.290198,23.077315,24.608947,22.023540
8,Croatia,2.600000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
9,Cyprus,2.500000,3.700000,2.400000,56.920863,27.422456,29.498128,6.165260,4.936008,1.229252,24.490862,12.426368,71.356113,13.881474,11.665777,3.096636


In [1395]:
# 1) Identify columns
metric_cols = [c for c in df_16.columns if c != 'country']
pct_cols    = [c for c in metric_cols if str(c).startswith('share_') or '(%)' in str(c)]
size_cols   = [c for c in metric_cols if str(c).startswith('avg_size')]

# 2) Normalize percentage columns to 0–100 NUMERIC (do NOT divide by 100)
for c in pct_cols:
    s = (df_16[c].astype(str)
                    .str.replace('%', '', regex=False)   # drop percent sign if present
                    .str.replace(',', '.', regex=False)  # handle decimal comma
                    .str.strip()
                    .replace({'': np.nan}))
    s = pd.to_numeric(s, errors='coerce')

    # If column appears to be 0–1 scale, scale UP to 0–100
    maxv = s.max(skipna=True)
    if pd.notna(maxv) and maxv <= 1.5:
        s = s * 100.0

    df_16[c] = s  # write back as float (e.g., 55.93)

# 3) Ensure avg_size columns stay numeric (unchanged values)
for c in size_cols:
    df_16[c] = pd.to_numeric(df_16[c], errors='coerce')

# 4) (Optional) add "(%)" to headers of percentage columns (header-only; no value changes)
df_16.rename(columns=lambda x: f'{x} (%)' if x in pct_cols and '(%)' not in str(x) else x, inplace=True)

# 5) Display with blanks instead of NaN (values remain numeric under the hood)
display(df_16.style.format(na_rep=''))


metric,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total (%),share_couple_with_children (%),share_couple_without_children (%),share_single_parent_total (%),share_single_mother (%),share_single_father (%),share_single_person (%),share_other_types (%),share_hh_0_children (%),share_hh_1_child (%),share_hh_2_children (%),share_hh_3plus_children (%)
0,Australia,2.527580,3.932863,2.775636,55.926052,29.904701,26.021351,10.373549,,,25.124159,8.576123,,,,
1,Austria,2.200000,3.800000,2.500000,48.927373,21.129493,27.797880,5.628256,4.779580,0.848677,38.337314,7.107057,77.781493,10.524053,8.574977,3.119476
2,Belgium,2.200000,3.900000,2.600000,52.219429,23.979842,28.239587,7.423107,6.077464,1.345643,35.502304,4.855161,73.974236,11.757354,10.153817,4.112671
3,Bulgaria,2.200000,3.500000,2.300000,40.303059,16.350410,23.952650,4.603051,3.875726,0.727325,35.809580,19.284309,78.210294,12.932273,7.480467,1.376966
4,Canada,2.425303,,,50.919441,25.300814,25.618627,8.715567,,,29.347961,11.017031,,,,
5,Chile,2.800000,,,,,,,,,,,,,,
6,Columbia,3.100732,,,,,,,,,,,,,,
7,Costa Rica,3.462513,4.372663,3.443867,52.441873,38.147069,14.294803,10.548101,9.489556,1.058545,11.270909,25.739118,30.290198,23.077315,24.608947,22.023540
8,Croatia,2.600000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
9,Cyprus,2.500000,3.700000,2.400000,56.920863,27.422456,29.498128,6.165260,4.936008,1.229252,24.490862,12.426368,71.356113,13.881474,11.665777,3.096636


In [1396]:
df_16.sample(10)

metric,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total (%),share_couple_with_children (%),share_couple_without_children (%),share_single_parent_total (%),share_single_mother (%),share_single_father (%),share_single_person (%),share_other_types (%),share_hh_0_children (%),share_hh_1_child (%),share_hh_2_children (%),share_hh_3plus_children (%)
14,Finland,1.90000,4.000000,2.600000,45.637840,17.056364,28.581477,5.427769,4.497123,0.930646,45.335962,3.598356,81.983051,7.888136,6.986441,3.142373
18,Hungary,2.30000,3.700000,2.400000,45.253981,20.432838,24.821143,7.073477,5.705424,1.368053,34.419371,13.253171,74.979597,13.156919,8.705330,3.158155
8,Croatia,2.60000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
0,Australia,2.52758,3.932863,2.775636,55.926052,29.904701,26.021351,10.373549,NaN,NaN,25.124159,8.576123,NaN,NaN,NaN,NaN
27,Luxembourg,2.30000,3.700000,2.500000,53.059577,26.859398,26.200179,6.631748,5.388820,1.242928,28.869818,11.438856,72.997552,12.486884,12.067156,2.413431
28,Malta,2.50000,3.700000,2.500000,46.920827,21.114001,25.806826,5.680812,4.557909,1.122903,32.512251,14.889819,76.489170,12.680505,7.806859,2.978339
21,Israel,3.19000,4.649476,2.863297,64.500000,45.800000,18.700000,9.100000,NaN,NaN,26.400000,NaN,NaN,NaN,NaN,NaN
15,France,2.10000,3.900000,2.600000,49.727114,22.188118,27.538996,7.676366,6.228670,1.447696,37.784050,4.812470,75.355517,11.426087,9.226449,3.991635
29,Mexico,3.56000,4.080000,2.760000,50.357352,39.016163,11.341189,11.216618,NaN,NaN,12.464293,25.961737,50.233880,22.833950,17.403083,9.529087
46,United States,2.51000,NaN,NaN,53.185494,19.846267,33.339226,6.797399,5.212952,1.584447,27.611972,12.405135,NaN,NaN,NaN,NaN


In [1397]:
df_16.columns.name = None
df_16.index.name = None

In [1398]:
df_16.sample(10)

,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,share_couple_total (%),share_couple_with_children (%),share_couple_without_children (%),share_single_parent_total (%),share_single_mother (%),share_single_father (%),share_single_person (%),share_other_types (%),share_hh_0_children (%),share_hh_1_child (%),share_hh_2_children (%),share_hh_3plus_children (%)
11,Denmark,1.900000,3.900000,2.500000,48.596980,20.408734,28.188133,6.308205,5.114202,1.194003,37.574211,7.520455,77.775278,10.541878,8.944527,2.738317
30,Netherlands,2.000000,3.900000,2.600000,53.601238,23.007511,30.593727,6.098421,5.004163,1.094258,38.504867,1.795475,78.652492,8.776867,9.272513,3.296971
19,Iceland,2.701931,4.119078,2.608646,45.194843,25.421669,19.773174,7.354279,6.231610,1.122668,29.161858,18.289020,NaN,NaN,NaN,NaN
43,Switzerland,2.214080,4.017677,2.584313,53.767682,24.046550,29.721030,4.698226,3.878421,0.819806,36.879491,4.654601,NaN,NaN,NaN,NaN
20,Ireland,2.400000,4.000000,2.700000,53.027121,29.449110,23.578011,6.925656,6.122436,0.803221,23.135621,16.911603,69.020563,12.418831,12.184343,6.376263
22,Italy,2.200000,3.600000,2.400000,46.704125,20.906322,25.797803,7.274459,5.649393,1.625066,36.638065,9.383350,77.789563,12.263053,8.282960,1.664424
18,Hungary,2.300000,3.700000,2.400000,45.253981,20.432838,24.821143,7.073477,5.705424,1.368053,34.419371,13.253171,74.979597,13.156919,8.705330,3.158155
12,EU average,2.262963,NaN,NaN,47.974518,22.090625,25.883877,6.707648,5.532233,1.175415,34.001383,11.316720,75.100758,12.283532,9.461296,3.151375
10,Czechia,2.300000,3.700000,2.400000,47.025112,21.703808,25.321304,7.154102,6.112638,1.041464,39.150397,6.670389,71.952265,13.852443,11.560185,2.635107
23,Japan,2.210988,3.854805,2.727759,45.073248,15.668254,29.404994,2.503514,2.257534,0.245980,37.969772,14.453466,81.938221,8.782047,7.165944,2.113788


In [1399]:
df_16.rename(columns=lambda c: re.sub(r'(?i)^share[_\s]*', '', c) if isinstance(c, str) else c, inplace=True)
df_16.drop_duplicates(inplace=True)
df_16.replace('', pd.NA, inplace=True)

metric_cols = [c for c in df_16.columns if c != 'country']

# A) Drop only rows where ALL metric columns are missing (safe):
df_16.dropna(subset=metric_cols, how='all', inplace=True)

# B) (optional) Also drop columns that are entirely missing:
df_16.dropna(axis=1, how='all', inplace=True)
df_16.isnull().sum()

country                                  0
avg_size_all                             2
avg_size_couple_with_children            8
avg_size_single_parent_with_children     8
couple_total (%)                         4
couple_with_children (%)                 5
couple_without_children (%)              5
single_parent_total (%)                  4
single_mother (%)                       11
single_father (%)                       11
single_person (%)                        4
other_types (%)                          5
hh_0_children (%)                       10
hh_1_child (%)                          10
hh_2_children (%)                       10
hh_3plus_children (%)                   10
dtype: int64

In [1400]:
# Drop exactly these columns and create df_16_general
cols_to_drop = [
    "avg_size_couple_with_children",
    "avg_size_single_parent_with_children",
    "single_mother (%)",
    "single_father (%)",
    "hh_0_children (%)",
    "hh_1_child (%)",
    "hh_2_children (%)",
    "hh_3plus_children (%)",
]

df_16_general = df_16.drop(columns=cols_to_drop, errors="ignore").copy()

# quick check (optional)
print(df_16_general.columns.tolist())

['country', 'avg_size_all', 'couple_total (%)', 'couple_with_children (%)', 'couple_without_children (%)', 'single_parent_total (%)', 'single_person (%)', 'other_types (%)']


In [1401]:
# Remove selected countries (exact matches)
df_16_general['country'] = df_16_general['country'].str.strip()  # trims spaces

countries_to_remove = ["Chile", "Columbia", "OECD average", "OECD-30 average", "OECD-36 average", "Israel"]
df_16_general = df_16_general[~df_16_general['country'].isin(countries_to_remove)].copy()


In [1402]:
df_16_general = df_16_general.reset_index(drop=True)
df_16_general

,country,avg_size_all,couple_total (%),couple_with_children (%),couple_without_children (%),single_parent_total (%),single_person (%),other_types (%)
0,Australia,2.527580,55.926052,29.904701,26.021351,10.373549,25.124159,8.576123
1,Austria,2.200000,48.927373,21.129493,27.797880,5.628256,38.337314,7.107057
2,Belgium,2.200000,52.219429,23.979842,28.239587,7.423107,35.502304,4.855161
3,Bulgaria,2.200000,40.303059,16.350410,23.952650,4.603051,35.809580,19.284309
4,Canada,2.425303,50.919441,25.300814,25.618627,8.715567,29.347961,11.017031
5,Costa Rica,3.462513,52.441873,38.147069,14.294803,10.548101,11.270909,25.739118
6,Croatia,2.600000,51.508875,24.776445,26.732361,5.422310,27.801580,15.267514
7,Cyprus,2.500000,56.920863,27.422456,29.498128,6.165260,24.490862,12.426368
8,Czechia,2.300000,47.025112,21.703808,25.321304,7.154102,39.150397,6.670389
9,Denmark,1.900000,48.596980,20.408734,28.188133,6.308205,37.574211,7.520455


In [1403]:
df_16_general.isnull().sum()

country                        0
avg_size_all                   0
couple_total (%)               0
couple_with_children (%)       0
couple_without_children (%)    0
single_parent_total (%)        0
single_person (%)              0
other_types (%)                0
dtype: int64

In [1404]:
drop_countries = ['OECD average', 'OECD-30 average', 'OECD-36 average', 'Canada', 'Chile', 'Columbia', 'United States','Iceland','Israel','EU average', 'New Zealand', 'Mexico', 'United Kingdom', 'Switzerland','Australia']

before = len(df_16)
df_16 = df_16[~df_16['country'].isin(drop_countries)].reset_index(drop=True)
print(f"Removed {before - len(df_16)} rows")

Removed 15 rows


In [1405]:
df_16.isnull().sum()

country                                 0
avg_size_all                            0
avg_size_couple_with_children           0
avg_size_single_parent_with_children    0
couple_total (%)                        0
couple_with_children (%)                0
couple_without_children (%)             0
single_parent_total (%)                 0
single_mother (%)                       0
single_father (%)                       0
single_person (%)                       0
other_types (%)                         0
hh_0_children (%)                       0
hh_1_child (%)                          0
hh_2_children (%)                       0
hh_3plus_children (%)                   0
dtype: int64

In [1406]:
df_16

,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,couple_total (%),couple_with_children (%),couple_without_children (%),single_parent_total (%),single_mother (%),single_father (%),single_person (%),other_types (%),hh_0_children (%),hh_1_child (%),hh_2_children (%),hh_3plus_children (%)
0,Austria,2.200000,3.800000,2.500000,48.927373,21.129493,27.797880,5.628256,4.779580,0.848677,38.337314,7.107057,77.781493,10.524053,8.574977,3.119476
1,Belgium,2.200000,3.900000,2.600000,52.219429,23.979842,28.239587,7.423107,6.077464,1.345643,35.502304,4.855161,73.974236,11.757354,10.153817,4.112671
2,Bulgaria,2.200000,3.500000,2.300000,40.303059,16.350410,23.952650,4.603051,3.875726,0.727325,35.809580,19.284309,78.210294,12.932273,7.480467,1.376966
3,Costa Rica,3.462513,4.372663,3.443867,52.441873,38.147069,14.294803,10.548101,9.489556,1.058545,11.270909,25.739118,30.290198,23.077315,24.608947,22.023540
4,Croatia,2.600000,3.900000,2.600000,51.508875,24.776445,26.732361,5.422310,4.386251,1.036059,27.801580,15.267514,74.183029,11.964128,10.095795,3.757049
5,Cyprus,2.500000,3.700000,2.400000,56.920863,27.422456,29.498128,6.165260,4.936008,1.229252,24.490862,12.426368,71.356113,13.881474,11.665777,3.096636
6,Czechia,2.300000,3.700000,2.400000,47.025112,21.703808,25.321304,7.154102,6.112638,1.041464,39.150397,6.670389,71.952265,13.852443,11.560185,2.635107
7,Denmark,1.900000,3.900000,2.500000,48.596980,20.408734,28.188133,6.308205,5.114202,1.194003,37.574211,7.520455,77.775278,10.541878,8.944527,2.738317
8,Estonia,1.800000,3.800000,2.600000,46.199713,25.464743,20.734971,6.828569,6.090750,0.737819,36.991570,9.980148,75.755102,12.530612,8.734694,2.979592
9,Finland,1.900000,4.000000,2.600000,45.637840,17.056364,28.581477,5.427769,4.497123,0.930646,45.335962,3.598356,81.983051,7.888136,6.986441,3.142373


In [1407]:
#df_16_general.to_csv('../data/Cleaned/general/cleaned_household_general.csv', index=False)

In [1408]:
df_16['country'] = df_16['country'].astype(str).str.strip()

# 2) Identify column groups
metric_cols = [c for c in df_16.columns if c != 'country']
pct_cols    = [c for c in metric_cols if '(%)' in str(c) or re.match(r'^share[_\s]', str(c) or '')]
size_cols   = [c for c in metric_cols if str(c).startswith('avg_size')]

# 3) Coerce to proper numeric types (DO NOT rescale percentages)
def to_float(s: pd.Series) -> pd.Series:
    return pd.to_numeric(
        s.astype(str)
         .str.replace('%', '', regex=False)   # strip a trailing % if any
         .str.replace(',', '.', regex=False)  # handle decimal comma
         .str.strip()
         .replace({'': np.nan}),
        errors='coerce'
    )

for c in pct_cols:
    df_16[c] = to_float(df_16[c])            # stays on 0–100 scale

for c in size_cols:
    df_16[c] = pd.to_numeric(df_16[c], errors='coerce')

# 4) Round to 2 decimals (all numeric metrics)
df_16[metric_cols] = df_16[metric_cols].round(2)

In [1409]:
df_16

,country,avg_size_all,avg_size_couple_with_children,avg_size_single_parent_with_children,couple_total (%),couple_with_children (%),couple_without_children (%),single_parent_total (%),single_mother (%),single_father (%),single_person (%),other_types (%),hh_0_children (%),hh_1_child (%),hh_2_children (%),hh_3plus_children (%)
0,Austria,2.20,3.80,2.50,48.93,21.13,27.80,5.63,4.78,0.85,38.34,7.11,77.78,10.52,8.57,3.12
1,Belgium,2.20,3.90,2.60,52.22,23.98,28.24,7.42,6.08,1.35,35.50,4.86,73.97,11.76,10.15,4.11
2,Bulgaria,2.20,3.50,2.30,40.30,16.35,23.95,4.60,3.88,0.73,35.81,19.28,78.21,12.93,7.48,1.38
3,Costa Rica,3.46,4.37,3.44,52.44,38.15,14.29,10.55,9.49,1.06,11.27,25.74,30.29,23.08,24.61,22.02
4,Croatia,2.60,3.90,2.60,51.51,24.78,26.73,5.42,4.39,1.04,27.80,15.27,74.18,11.96,10.10,3.76
5,Cyprus,2.50,3.70,2.40,56.92,27.42,29.50,6.17,4.94,1.23,24.49,12.43,71.36,13.88,11.67,3.10
6,Czechia,2.30,3.70,2.40,47.03,21.70,25.32,7.15,6.11,1.04,39.15,6.67,71.95,13.85,11.56,2.64
7,Denmark,1.90,3.90,2.50,48.60,20.41,28.19,6.31,5.11,1.19,37.57,7.52,77.78,10.54,8.94,2.74
8,Estonia,1.80,3.80,2.60,46.20,25.46,20.73,6.83,6.09,0.74,36.99,9.98,75.76,12.53,8.73,2.98
9,Finland,1.90,4.00,2.60,45.64,17.06,28.58,5.43,4.50,0.93,45.34,3.60,81.98,7.89,6.99,3.14


In [1410]:
#df_16.to_csv('../data/Cleaned/cleaned_household_oecd.csv', index=False)

In [1411]:
#df_16.to_sql('household_oecd', engine, if_exists= 'replace', index= False)

In [1415]:
df_17 = pd.read_csv('../data/Raw/OECD/Households-by-type,-presence-of-children-and-country,-2015-2024.csv')
df_17

,Category,Single adult with children,Single adult without children,Couple with children,Couple without children,Other type of household with children,Other type of household without children
0,2015,6147.3,64181.3,31679.8,46641.6,11698.9,30771.6
1,2016,6148.5,63891.1,31907.3,47308.2,11766.3,30559.5
2,2017,6108.5,65353.9,32091.5,47426.1,11530.2,30297.5
3,2018,6163.6,66165.5,31720.2,48194.8,11342.5,30224.0
4,2019,6246.4,67417.9,31710.1,48503.6,11285.7,30134.8
5,2020,6136.4,67412.9,31622.2,48831.2,11212.9,30445.2
6,2021,5691.9,70200.4,30558.3,47447.4,11611.8,30700.7
7,2022,5984.9,72134.3,30469.3,47995.5,11513.6,30412.1
8,2023,5924.8,73396.2,30313.0,48477.5,11443.5,30608.8
9,2024,6077.7,75049.7,30286.5,49058.4,11311.9,30487.3


In [1416]:
df_18 = pd.read_csv('../data/Raw/OECD/Households-with-children-by-number-of-children,-2024.csv')
df_18

,Category,1 child,2 children,3 children or more
0,European Union,11.7,8.9,3.0
1,NaN,NaN,NaN,NaN
2,Slovakia,17.1,14.5,4.0
3,Ireland,12.4,12.2,6.4
4,Cyprus,13.9,11.7,3.1
5,Czechia,13.9,11.6,2.6
6,Romania,14.3,9.2,4.0
7,Luxembourg,12.5,12.1,2.4
8,Belgium,11.8,10.2,4.1
9,Croatia,12.0,10.1,3.8
